# Imports

In [2]:
import pandas as pd
import numpy as np
import json
import glob
from datetime import datetime
from util import get_category

# Get CSV files

In [46]:
def read_simplii_files():
    # get statement folder path
    config = open("./config.json").read()
    folder_path = json.loads(config).get("simplii").get("folderPath")
    csv_files = glob.glob(f"{folder_path}/*.csv")
    dfs = {}
    combined_df = pd.DataFrame()
    for csv_file in csv_files:
        print(csv_file)
        df = pd.read_csv(
            csv_file,
            index_col=False,
        )
        if len(df.columns) == 5:
            df = df.set_axis(
                ["Date", "Transaction Details", "Funds Out", "Funds In", "Account"],
                axis=1,
            )
        else:
            df = df.set_axis(
                ["Date", "Transaction Details", "Funds Out", "Funds In"], axis=1
            )
            df["Credit Account"] = "*" * 16
        if "visa" in csv_file.lower():
            df = df[~df["Funds In"].notnull()]
        df["sourceFile"] = csv_file
        dfs[csv_file] = df
        combined_df = pd.concat([combined_df, df])
    combined_df = combined_df.replace({np.nan: None})
    combined_df[["Month", "Day", "Year"]] = combined_df["Date"].str.split(
        "/", expand=True
    )
    combined_df["Category"] = combined_df["Transaction Details"].apply(
        lambda x: get_category(x.lower())
    )
    combined_df["Points"] = 0
    combined_df = combined_df[
        [
            "Account",
            "Date",
            "Month",
            "Day",
            "Year",
            "Category",
            "Transaction Details",
            "Funds In",
            "Funds Out",
            "Points",
            "sourceFile",
        ]
    ]
    return combined_df

In [47]:
combined_df = read_simplii_files()
combined_df.head()

C:\Users\Nielson\Documents\Statements\simplii\SIMPLII_chequeing.csv
C:\Users\Nielson\Documents\Statements\simplii\SIMPLII_visa.csv


,Account,Date,Month,Day,Year,Category,Transaction Details,Funds In,Funds Out,Points,sourceFile
0,None,05/15/2024,05,15,2024,Transfers,INTERAC E-TRANSFER RECEIVE NIELSON TRUNG,10.0,None,0,C:\Users\Nielson\Documents\Statements\simplii\...
1,None,05/15/2024,05,15,2024,Transfers,INTERAC E-TRANSFER RECEIVE NIELSON TRUNG,1000.0,None,0,C:\Users\Nielson\Documents\Statements\simplii\...
2,None,05/29/2024,05,29,2024,Transfers,INTERAC E-TRANSFER RECEIVE SHAHANAH LIGARAY,22.0,None,0,C:\Users\Nielson\Documents\Statements\simplii\...
3,None,06/03/2024,06,03,2024,Transfers,INTERAC E-TRANSFER RECEIVE RYAN PANG,28.0,None,0,C:\Users\Nielson\Documents\Statements\simplii\...
4,None,06/03/2024,06,03,2024,Transfers,INTERAC E-TRANSFER RECEIVE WESLEY TZE FAH LAI,30.0,None,0,C:\Users\Nielson\Documents\Statements\simplii\...


# Funds In & Out per Month

In [48]:
groupby_df = (
    combined_df[["Year", "Month", "Funds In", "Funds Out"]]
    .groupby(["Year", "Month"])
    .sum()
)
groupby_df

Funds In Funds Out
Year Month                   
2024 05      1032.0     54.28
     06     4365.52    471.15
     07     4473.03   5553.34
     08      2624.0   2602.58

# Get Monthly Spending per Category

In [49]:
# combined_df[combined_df["Category"] == "Restaurant"][["Year","Month","Funds Out", "Funds In", "Category"]].groupby(["Year","Month","Category"]).sum()
combined_df[["Year", "Month", "Funds In", "Funds Out", "Category"]].groupby(
    ["Year", "Month", "Category"]
).sum().sort_values(["Year", "Month", "Category"], ascending=True)

Funds In Funds Out
Year Month Category                     
2024 05    Restaurant        0     54.28
           Transfers    1032.0         0
     06    Deposit      4210.0         0
           Groceries         0     92.93
           Other          0.02     51.39
           Restaurant        0    326.83
           Transfers     155.5         0
     07    Deposit     4209.99         0
           Investment        0    3100.0
           Other          0.04         0
           Payment           0   1478.15
           Restaurant        0    416.32
           Transfers     263.0    558.87
     08    Deposit      2105.0         0
           Investment        0     500.0
           Payment           0   1560.24
           Restaurant        0     42.34
           Transfers     119.0     500.0
           Withdrawal    400.0         0

In [50]:
sort_by_cols = ["Date", "Category", "Transaction Details"]
combined_df[
    combined_df["Funds Out"].notnull()
    & ~combined_df["Category"].isin(["Transfers", "Investment", "Deposit", "Payment"])
].sort_values(by=sort_by_cols, ascending=False)

,Account,Date,Month,Day,Year,Category,Transaction Details,Funds In,Funds Out,Points,sourceFile
2,4525********3980,08/01/2024,08,01,2024,Restaurant,"SNOWY VILLAGE CAFE CALGARY, AB",None,8.93,0,C:\Users\Nielson\Documents\Statements\simplii\...
3,4525********3980,08/01/2024,08,01,2024,Restaurant,"SNOWY VILLAGE CAFE CALGARY, AB",None,13.39,0,C:\Users\Nielson\Documents\Statements\simplii\...
1,4525********3980,08/01/2024,08,01,2024,Restaurant,"Five Guys 1963 Calgary, AB",None,20.02,0,C:\Users\Nielson\Documents\Statements\simplii\...
4,4525********3980,07/31/2024,07,31,2024,Restaurant,"DONAIR PALACE CALGARY, AB",None,10.49,0,C:\Users\Nielson\Documents\Statements\simplii\...
5,4525********3980,07/29/2024,07,29,2024,Restaurant,"TSUJIRI CALGARY CALGARY, AB",None,17.04,0,C:\Users\Nielson\Documents\Statements\simplii\...
7,4525********3980,07/29/2024,07,29,2024,Restaurant,"MR. PRETZELS ROCKY VIEW CO, AB",None,5.76,0,C:\Users\Nielson\Documents\Statements\simplii\...
6,4525********3980,07/29/2024,07,29,2024,Restaurant,"HCM*KINJO DALHOUSIE K2 ST CALGARY, AB",None,41.14,0,C:\Users\Nielson\Documents\Statements\simplii\...
8,4525********3980,07/26/2024,07,26,2024,Restaurant,"SAIGON REX CALGARY, AB",None,44.08,0,C:\Users\Nielson\Documents\Statements\simplii\...
9,4525********3980,07/22/2024,07,22,2024,Restaurant,"CHEONG DAM KOREAN BBQ CALGARY, AB",None,115.76,0,C:\Users\Nielson\Documents\Statements\simplii\...
10,4525********3980,07/18/2024,07,18,2024,Restaurant,"TIM HORTONS #20024 CALGARY, AB",None,20.13,0,C:\Users\Nielson\Documents\Statements\simplii\...


# Payments Bills

In [51]:
combined_df[combined_df["Category"] == "Payment"][
    ["Year", "Month", "Category", "Funds In", "Funds Out"]
].groupby(["Year", "Month", "Category"]).sum()

Funds In Funds Out
Year Month Category                   
2024 07    Payment         0   1478.15
     08    Payment         0   1560.24

In [52]:
combined_df[combined_df["Category"] == "Payment"].sort_values(
    by=["Date"], ascending=False
)

,Account,Date,Month,Day,Year,Category,Transaction Details,Funds In,Funds Out,Points,sourceFile
30,None,08/02/2024,08,02,2024,Payment,MISCELLANEOUS PAYMENTS American Express CHEQUE,None,1560.24,0,C:\Users\Nielson\Documents\Statements\simplii\...
26,None,07/31/2024,07,31,2024,Payment,MISCELLANEOUS PAYMENTS CIBC CARD PRODUCTS DIVI...,None,100.92,0,C:\Users\Nielson\Documents\Statements\simplii\...
22,None,07/19/2024,07,19,2024,Payment,EFT DEBIT AMEX BANK OF CANADA CHEQUE,None,1000.42,0,C:\Users\Nielson\Documents\Statements\simplii\...
15,None,07/05/2024,07,05,2024,Payment,MISCELLANEOUS PAYMENTS CIBC CARD PRODUCTS DIVI...,None,292.65,0,C:\Users\Nielson\Documents\Statements\simplii\...
13,None,07/04/2024,07,04,2024,Payment,MISCELLANEOUS PAYMENTS CIBC CARD PRODUCTS DIVI...,None,84.16,0,C:\Users\Nielson\Documents\Statements\simplii\...


In [53]:
def get_simplii_transactions():
    df = read_simplii_files()
    df = df[~df["description"].str.contains("PAYMENT")]
    df["amount"] = df["amount"].abs()
    df["amount"] *= -1
    df["foreignAmount"] = df["foreignAmount"].fillna(0)
    df["points"] = df["points"].fillna(0)
    df[["month", "day", "year"]] = df["date"].str.split("/", expand=True)
    df["date"] = df["year"] + "/" + df["month"] + "/" + df["day"]
    df["id"] = df["id"].str.replace("Reference: ", "")
    df["account"] = "simplii"
    df["description"] = df["description"].str.lower()
    df["description"] = df["description"].str.replace("-", "")
    df["category"] = df["description"].apply(get_category)
    df["sourceFile"] = input_file
    df = df.drop("foreignAmount", axis=1)
    columns = [
        "account",
        "date",
        "year",
        "month",
        "day",
        "category",
        "description",
        "amount",
        "points",
        "sourceFile",
    ]
    df = df[columns]
    transactions = df.values.tolist()
    return transactions